In [15]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_classif
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix,recall_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

In [16]:
df = pd.read_csv('modified_dfv3-1_filtered.csv')

In [17]:
# Assuming the 'dateTimeTransaction' column is already converted to datetime format
# If not, you should convert it first
df['dateTimeTransaction'] = pd.to_datetime(df['dateTimeTransaction'])

# Extracting date-related features
df['year'] = df['dateTimeTransaction'].dt.year
df['month'] = df['dateTimeTransaction'].dt.month
df['day_name'] = df['dateTimeTransaction'].dt.day_name()
df['hour'] = df['dateTimeTransaction'].dt.hour


In [18]:
import numpy as np
import pandas as pd
import sklearn
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report,accuracy_score
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM
from pylab import rcParams
rcParams['figure.figsize'] = 14, 8
RANDOM_SEED = 42
LABELS = ["Normal", "Fraud"]

In [19]:
## Get the Fraud and the normal dataset

fraud = df[df['is_fraud']==1]

normal = df[df['is_fraud']==0]

In [20]:
df_copy=df.copy()
print(df_copy.head())

   encryptedHexCardNo dateTimeTransaction  \
0        5.359540e+15 2020-09-25 21:36:00   
1        3.513620e+15 2020-11-15 04:58:00   
2        3.023540e+13 2020-11-13 12:15:00   
3        6.011100e+15 2020-06-29 08:55:00   
4        3.537800e+15 2020-07-18 06:56:00   

                     merchantCategoryCode  transactionAmount  latitude  \
0                    fraud_Mosciski Group            8517.38   45.7801   
1                   fraud_Dooley-Thompson             187.99   43.7588   
2                    fraud_Roberts-Beahan               7.81   38.5072   
3                         fraud_Kunze Inc             122.29   36.9688   
4  fraud_Greenholt, O'Hara and Balistreri               9.21   47.4852   

   longitude  cardBalance  is_fraud  year  month  day_name  hour  
0  -111.1439     65597.94         0  2020      9    Friday    21  
1   -97.8712     42507.98         0  2020     11    Sunday     4  
2   -81.8900     16664.64         0  2020     11    Friday    12  
3   -78.5615    

In [21]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import pickle

# Sample DataFrame creation or load your data here


# Initialize separate label encoders for each categorical feature
le_day_name = LabelEncoder()
le_merchant = LabelEncoder()

# Fit the label encoders to the respective columns
df_copy['day_name'] = le_day_name.fit_transform(df['day_name'])
df_copy['merchantCategoryCode'] = le_merchant.fit_transform(df['merchantCategoryCode'])

# Save the label encoders to .pkl files
with open('day_name_encoder.pkl', 'wb') as file:
    pickle.dump(le_day_name, file)

with open('merchant_encoder.pkl', 'wb') as file:
    pickle.dump(le_merchant, file)


In [22]:

Fraud = df_copy[df['is_fraud']==1]

Valid = df_copy[df['is_fraud']==0]

outlier_fraction = len(Fraud)/float(len(Valid))

In [23]:
#Create independent and Dependent Features
columns = df_copy.columns.tolist()
# Filter the columns to remove data we do not want
columns = [c for c in columns if c not in ["is_fraud","dateTimeTransaction"]]
# Store the variable we are predicting
target = "is_fraud"
# Define a random state
state = np.random.RandomState(42)
X = df_copy[columns]
Y = df_copy[target]
X_outliers = state.uniform(low=0, high=1, size=(X.shape[0], X.shape[1]))
# Print the shapes of X & Y
print(X.shape)
print(X.head())
print(Y.shape)
print(X.columns)

(10000, 10)
   encryptedHexCardNo  merchantCategoryCode  transactionAmount  latitude  \
0        5.359540e+15                   441            8517.38   45.7801   
1        3.513620e+15                   142             187.99   43.7588   
2        3.023540e+13                   521               7.81   38.5072   
3        6.011100e+15                   352             122.29   36.9688   
4        3.537800e+15                   209               9.21   47.4852   

   longitude  cardBalance  year  month  day_name  hour  
0  -111.1439     65597.94  2020      9         0    21  
1   -97.8712     42507.98  2020     11         3     4  
2   -81.8900     16664.64  2020     11         0    12  
3   -78.5615     39371.41  2020      6         1     8  
4  -120.4158     38967.53  2020      7         2     6  
(10000,)
Index(['encryptedHexCardNo', 'merchantCategoryCode', 'transactionAmount',
       'latitude', 'longitude', 'cardBalance', 'year', 'month', 'day_name',
       'hour'],
      dtype='o

In [24]:
import numpy as np
import pickle
from sklearn.metrics import classification_report, accuracy_score
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

In [25]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, InputLayer
from tensorflow.keras.optimizers import Adam

def fit_autoencoder(X_train, epochs=50, batch_size=32):
    # Define the autoencoder architecture
    autoencoder = Sequential([
        InputLayer(input_shape=(X_train.shape[1],)),
        Dense(128, activation='relu'),
        Dense(64, activation='relu'),
        Dense(128, activation='relu'),
        Dense(X_train.shape[1], activation='sigmoid')
    ])
    
    # Compile the autoencoder
    autoencoder.compile(optimizer=Adam(), loss='mse')
    
    # Train the autoencoder
    history = autoencoder.fit(X_train, X_train, epochs=epochs, batch_size=batch_size, validation_split=0.2)
    
    return autoencoder, history


In [27]:
from sklearn.metrics import mean_squared_error

def get_reconstruction_error(model, X_test):
    # Predict the outputs with the autoencoder
    reconstructed = model.predict(X_test)
    
    # Calculate MSE for each sample
    mse = np.mean(np.power(X_test - reconstructed, 2), axis=1)
    
    return mse


In [28]:
from imblearn.under_sampling import RandomUnderSampler
 
from sklearn.datasets import make_classification
from imblearn.under_sampling import RandomUnderSampler

# Initialize the undersampler
rus = RandomUnderSampler(random_state=42)

# Resample the dataset
X_res, Y_res = rus.fit_resample(X, Y)

# Split the resampled data for training and validation (if not done so already)
X_train, X_test, Y_train, Y_test = train_test_split(X_res, Y_res, test_size=0.2, random_state=42)

# Fit the autoencoder on the resampled training data
autoencoder, _ = fit_autoencoder(X_train, epochs=50, batch_size=32)

# Calculate reconstruction errors on the test set
reconstruction_errors = get_reconstruction_error(autoencoder, X_test)

# Setting the anomaly detection threshold
outlier_fraction = 0.01  # Example fraction of outliers
threshold = np.percentile(reconstruction_errors, 100 * outlier_fraction)
y_pred_autoencoder = np.where(reconstruction_errors > threshold, 1, 0)

# Evaluation on the test set
print("Autoencoder: Number of errors = {}".format((y_pred_autoencoder != Y_test).sum()))
print("Accuracy Score :")
print(accuracy_score(Y_test, y_pred_autoencoder))
print("Classification Report :")
print(classification_report(Y_test, y_pred_autoencoder))



Epoch 1/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 87268781139829168139857746030755840.0000 - val_loss: 123289280919240906527272750664384512.0000
Epoch 2/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 80070565855717567446049505580941312.0000 - val_loss: 123289280919240906527272750664384512.0000
Epoch 3/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 61075539616520335385180849540431872.0000 - val_loss: 123289280919240906527272750664384512.0000
Epoch 4/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 119376964543725590687776952817811456.0000 - val_loss: 123289280919240906527272750664384512.0000
Epoch 5/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 101665399874938340905276011597266944.0000 - val_loss: 123289280919240906527272750664384512.0000
Epoch 6/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 87268820753910425272026542802731008.0000 - val_loss: 123289280919240906527272750664384512.0000
Epoch 7/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1193770536754084192351567455547555

In [29]:
from tensorflow.keras.metrics import MeanSquaredError

autoencoder.compile(optimizer='adam', loss='mse', metrics=[MeanSquaredError()])


In [30]:
import tensorflow as tf
from tensorflow.keras.utils import register_keras_serializable

@register_keras_serializable()
def mse(y_true, y_pred):
    return tf.reduce_mean(tf.square(y_true - y_pred))

# Use this custom 'mse' when compiling your model
autoencoder.compile(optimizer='adam', loss='mse', metrics=[mse])


In [31]:
import tensorflow as tf
from tensorflow.keras.utils import register_keras_serializable
from tensorflow.keras.models import load_model

@register_keras_serializable()
def mse(y_true, y_pred):
    return tf.reduce_mean(tf.square(y_true - y_pred))

# Load the model with custom objects specified
loaded_model = load_model('autoencoder_model.h5', custom_objects={'mse': mse})

TypeError: Could not locate function 'mse'. Make sure custom classes are decorated with `@keras.saving.register_keras_serializable()`. Full object config: {'module': 'keras.metrics', 'class_name': 'function', 'config': 'mse', 'registered_name': 'mse'}

In [36]:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
autoencoder.save('autoencoder_model.h5')  # Saves the model to an HDF5 file
